In [2]:
import numpy as np
from scipy.optimize import curve_fit
import torch
import torch.nn as nn
import torch.masked as masked
import torch.distributions as ptd


In [3]:
val_dim = 7
act_dim = 4
obs_dim = 5
n_layers = 2
layer_size = 64

""" sooo masked arrays dont have matrix multiplication 
autograd support. so we need to completely pivot to the sparse library 
and instead use COO tensors, stored in either CSR or CSC format but IDK 
the difference... """

np.set_printoptions(precision=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def np2torch(x, requires_grad=False, cast_double_to_float=True):
    if isinstance(x, np.ma.MaskedArray):
        mask = torch.from_numpy(~x.mask).to(device)
        x = torch.from_numpy(x.data).to(device)
        if requires_grad: x = x.float()
        x = masked.as_masked_tensor(x, mask).to(device)
    elif isinstance(x, np.ndarray):
        x = torch.from_numpy(x).to(device)
    else:
        x = torch.Tensor(x).to(device)
    if requires_grad:
        x.requires_grad = True
    elif cast_double_to_float and x.dtype == torch.float64:
        x = x.float()
    return x

def torch2np(x: torch.Tensor | masked.MaskedTensor):
    if isinstance(x, masked.MaskedTensor):
        mask = ~x._masked_mask.detach().cpu().numpy()
        data = x._masked_data.detach().cpu().numpy()
        return np.ma.MaskedArray(data, mask=mask)
    return x.detach().cpu().numpy()

class MaskedSequential(nn.Sequential):
    """ Version of nn.Sequential that can handle masked tensors """
    def __init__(self, *args):
        super().__init__(*args)
    
    def forward(self, x):
        isMasked = isinstance(x, masked.MaskedTensor)
        if isMasked:
            mask = x._masked_mask[...,:1].to(device)
            req_grad = x.requires_grad
            x    = x._masked_data.to(device).requires_grad_(req_grad)
        for module in self:
            x = module(x)
        if isMasked:
            # account for different output, input dims
            newshape = [1]*(len(x.shape)-1)+[x.shape[-1]]
            mask = mask.repeat(*newshape).detach()
            return x, mask
        return x

def build_mlp(input_size, output_size, n_layers, hidden_size, activation=nn.ReLU()):
    """ Build multi-layer perception with n_layers hidden layers of size hidden_size """
    layers = [nn.Linear(input_size,hidden_size),activation]
    for i in range(n_layers-1):
        layers.append(nn.Linear(hidden_size,hidden_size))
        layers.append(activation)
    layers.append(nn.Linear(hidden_size, output_size))
    return MaskedSequential(*layers)

class BaselineNetwork(nn.Module):
    """
    Class for implementing Baseline network
    """
    def __init__(self):
        super().__init__()
        self.network = build_mlp(val_dim, 1, n_layers, layer_size)
        self.optimizer = torch.optim.Adam(params=self.network.parameters(),lr=1e-3)
    
    def forward(self, observations: torch.Tensor | torch.masked.MaskedTensor):
        if isinstance(observations, masked.MaskedTensor):
            """ implement a masked tensor squeeze """
            print('forward!')
            values, mask = self.network(observations)
            req_grad = values.requires_grad
            values = values.squeeze().clone().detach()
            mask   = mask.squeeze()
            return masked.masked_tensor(values, mask, requires_grad=req_grad).to(device)
        return self.network(observations).squeeze()

    def calculate_advantage(self, returns: np.ndarray | np.ma.MaskedArray, observations: np.ndarray | np.ma.MaskedArray) -> np.ndarray | np.ma.MaskedArray:
        """ Compute advantages given returns """
        return returns - torch2np(self.forward(np2torch(observations)))

    def update_baseline(self, returns: np.ndarray, observations: np.ndarray):
        """ Gradient baseline to match returns """
        returns = np2torch(returns, True)
        print('rets')
        observations = np2torch(observations, True)
        print('obs')
        print(returns._masked_data.requires_grad, observations._masked_data.requires_grad)
        baseline = self.forward(observations)
        print(baseline._masked_data.requires_grad)
        self.optimizer.zero_grad()
        loss = torch.mean((baseline - returns)**2)
        loss.backward()
        self.optimizer.step()

nb = 10
nt = 20
baseline = BaselineNetwork()
v = np.ma.empty((nb, nt, val_dim))
v[:] = np.ma.masked
r = np.ma.empty((nb, nt))
r[:] = np.ma.masked
for i in range(nb):
    length = int(np.random.rand(1)[0]*nt)
    v[i, :length, :] = np.random.rand(length, val_dim)
    r[i, :length] = np.random.rand(length)
print(r.shape, v.shape)
#advantages = baseline.calculate_advantage(r, v)
#print(advantages.shape)
#print(advantages)
#for i in range(10):
#    baseline.update_baseline(r, v)



(10, 20) (10, 20, 7)
forward!
rets
obs
False False
forward!
False
rets
obs
False False
forward!
False
rets
obs
False False
forward!
False
rets
obs
False False
forward!
False
rets
obs
False False
forward!
False
rets
obs
False False
forward!
False
rets
obs
False False
forward!
False
rets
obs
False False
forward!
False
rets
obs
False False
forward!
False
rets
obs
False False
forward!
False


/opt/miniconda3/envs/rl_market/lib/python3.12/site-packages/torch/masked/maskedtensor/core.py:156: UserWarning: The PyTorch API of MaskedTensors is in prototype stage and will change in the near future. Please open a Github issue for features requests and see our documentation on the torch.masked module for further information about the project.
  warnings.warn(("The PyTorch API of MaskedTensors is in prototype stage "
/opt/miniconda3/envs/rl_market/lib/python3.12/site-packages/torch/masked/maskedtensor/core.py:161: UserWarning: It is not recommended to create a MaskedTensor with a tensor that requires_grad. To avoid this, you can use data.clone().detach()
  warnings.warn("It is not recommended to create a MaskedTensor with a tensor that requires_grad. "


In [43]:
val_dim = 7
act_dim = 4
obs_dim = 5
n_layers = 2
layer_size = 64

""" masked arrays are of the dimension 
nbatch x ntime x nfeatures, where ntime varies across batches. 
can we implement a spare tensor version of this that might be more efficient? yes. will I? no. """

np.set_printoptions(precision=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def np2torch(x, requires_grad=False, cast_double_to_float=True):
    if isinstance(x, np.ma.MaskedArray):
        mask = torch.from_numpy(~x.mask).to(device)
        x = torch.from_numpy(x.data).to(device)
        if requires_grad: x = x.float()
        x = masked.as_masked_tensor(x, mask).to(device)
    elif isinstance(x, np.ndarray):
        x = torch.from_numpy(x).to(device)
    else:
        x = torch.Tensor(x).to(device)
    if requires_grad:
        x.requires_grad = True
    elif cast_double_to_float and x.dtype == torch.float64:
        x = x.float()
    return x

def torch2np(x: torch.Tensor | masked.MaskedTensor):
    if isinstance(x, masked.MaskedTensor):
        mask = ~x._masked_mask.detach().cpu().numpy()
        data = x._masked_data.detach().cpu().numpy()
        return np.ma.MaskedArray(data, mask=mask)
    return x.detach().cpu().numpy()

class MaskedSequential(nn.Sequential):
    """ Version of nn.Sequential that can handle masked tensors """
    def __init__(self, *args):
        super().__init__(*args)
    
    def forward(self, x):
        isMasked = isinstance(x, masked.MaskedTensor)
        if isMasked:
            mask = x._masked_mask[...,:1].to(device)
            req_grad = x.requires_grad
            x    = x._masked_data.to(device).requires_grad_(req_grad)
        for module in self:
            x = module(x)
        if isMasked:
            # account for different output, input dims
            newshape = [1]*(len(x.shape)-1)+[x.shape[-1]]
            mask = mask.repeat(*newshape).detach()
            return x, mask
        return x

def build_mlp(input_size, output_size, n_layers, hidden_size, activation=nn.ReLU()):
    """ Build multi-layer perception with n_layers hidden layers of size hidden_size """
    layers = [nn.Linear(input_size,hidden_size),activation]
    for i in range(n_layers-1):
        layers.append(nn.Linear(hidden_size,hidden_size))
        layers.append(activation)
    layers.append(nn.Linear(hidden_size, output_size))
    return MaskedSequential(*layers)

class BaselineNetwork(nn.Module):
    """
    Class for implementing Baseline network
    """
    def __init__(self):
        super().__init__()
        self.network = build_mlp(val_dim, 1, n_layers, layer_size)
        self.optimizer = torch.optim.Adam(params=self.network.parameters(),lr=1e-3)
    
    def forward(self, observations: torch.Tensor | torch.masked.MaskedTensor):
        if isinstance(observations, masked.MaskedTensor):
            """ implement a masked tensor squeeze """
            print('forward!')
            values, mask = self.network(observations)
            req_grad = values.requires_grad
            values = values.squeeze().clone().detach()
            mask   = mask.squeeze()
            return masked.masked_tensor(values, mask, requires_grad=req_grad).to(device)
        return self.network(observations).squeeze()

    def calculate_advantage(self, returns: np.ndarray | np.ma.MaskedArray, observations: np.ndarray | np.ma.MaskedArray) -> np.ndarray | np.ma.MaskedArray:
        """ Compute advantages given returns """
        return returns - torch2np(self.forward(np2torch(observations)))

    def update_baseline(self, returns: np.ndarray, observations: np.ndarray):
        """ Gradient baseline to match returns """
        returns = np2torch(returns, True)
        print('rets')
        observations = np2torch(observations, True)
        print('obs')
        print(returns._masked_data.requires_grad, observations._masked_data.requires_grad)
        baseline = self.forward(observations)
        print(baseline._masked_data.requires_grad)
        self.optimizer.zero_grad()
        loss = torch.mean((baseline - returns)**2)
        loss.backward()
        self.optimizer.step()

def get_lengths(arr: np.ma.MaskedArray) -> np.ndarray:
    """ print length of the true elements in arr """
    lengths = np.empty(arr.shape[0], dtype=int)
    for i in range(arr.shape[0]):  # get last non-masked value in each row
        trues = np.where(~arr.mask[i])[0]
        if len(trues.shape) > 1:
            trues = trues[:,0]  # we dont care about obs_dim
        if not len(trues): 
            lengths[i] = 0
            continue
        lengths[i] = trues[-1]
    return lengths

class GaussianPolicy(nn.Module):
    def __init__(self, network, action_dim):
        nn.Module.__init__(self)
        self.network = network
        self.log_std = nn.Parameter(torch.zeros(action_dim), requires_grad=False)

    def std(self):
        """ Returns: torch.Tensor of shape [dim(action space)] """
        return torch.exp(self.log_std)

    def log_probs(self, distribution: ptd.Distribution, actions: torch.Tensor | masked.MaskedTensor):
        """ Return log probs of actions """
        if isinstance(actions, masked.MaskedTensor):
            data = torch.nan_to_num(actions._masked_data, 1)
            mask = actions._masked_mask[...,0]  # bc probs are 1D
            req_grad = actions.requires_grad
            probs = distribution.log_prob(data)
            return masked.masked_tensor(probs.detach(), mask.detach(), requires_grad=req_grad)
        return distribution.log_prob(actions)

    def entropy(self, distribution: ptd.Distribution, observations: torch.Tensor | masked.MaskedTensor):
        """ Return entropy of the distribution """
        entropy = distribution.entropy()
        if not isinstance(observations, masked.MaskedTensor):
            return entropy     # bc entropy is 1D
        mask = observations._masked_mask[...,0]
        req_grad = observations.requires_grad
        return masked.masked_tensor(entropy.detach(), mask.detach(), requires_grad=req_grad)

    def act(self, observations: np.ndarray | np.ma.MaskedArray, return_log_prob = False):
        """ Return np.ndarray actions (and log probs)? from action distribution """
        observations = np2torch(observations)
        distribution = self.action_distribution(observations)
        actions = distribution.sample()
        sampled_actions = torch2np(actions)
        if return_log_prob:
            log_probs = torch2np(self.log_probs(distribution, actions))
            return sampled_actions, log_probs
        return sampled_actions

    def action_distribution(self, observations):
        """ continuous action distribution for given observations """
        means = self.network(observations)
        if isinstance(means, masked.MaskedTensor):
            means = means._masked_data.nan_to_num(0)
        return ptd.MultivariateNormal(means, scale_tril=torch.diag(self.std()))

class PolicyGradient():
    """
    Class for implementing a policy gradient algorithm
    """
    def __init__(self) -> None:
        """
        Initialize Policy Gradient Class
            - config: class with all parameters
        """
        self.discount = 0.99
        self.lambd = 0.95
        # set the baseline (value) network (val_dim -> 1)
        self.baseline = BaselineNetwork()
        self.eps_clip = 0.9
        self.do_clip = True
        self.entropy_coef = 0.02
        self.do_ppo = True
        # option to intialize policy on startup? why not
        network = build_mlp(obs_dim, act_dim, n_layers, layer_size)
        self.policy = GaussianPolicy(network, act_dim)
        self.optimizer = torch.optim.Adam(params=self.policy.parameters(),lr=1e-3)

    def get_finals(self, returns: np.ndarray | np.ma.MaskedArray) -> np.ndarray:
        """ Get final returns from batched returns of shape (nb x nt) """
        if not isinstance(returns, np.ma.MaskedArray):
            return returns[:,-1]
        finals = np.empty(returns.shape[0])
        for i in range(returns.shape[0]):  # get last non-masked value in each row
            trues = np.where(~returns.mask[i])[0]
            if not len(trues):
                finals[i] = 0
                continue
            finals[i] = returns[i, trues[-1]]
        return finals

    def get_returns(self, rewards: np.ndarray | np.ma.MaskedArray) -> np.ndarray | np.ma.MaskedArray:
        """ Classic returns from batched rewards of shape (nb x nt) """
        isMasked = isinstance(rewards, np.ma.MaskedArray)
        returns = np.empty_like(rewards)
        if isMasked:
            mask = rewards.mask
            rewards = rewards.filled(0)   # we want to add when OR on the masks
        returns[:, -1] = rewards[:, -1]
        for t in reversed(range(rewards.shape[1]-1)):
            returns[:, t] = rewards[:, t] + self.discount*rewards[:, t+1]
        if isMasked:
            returns = np.ma.masked_array(returns, mask)
        return returns
    
    def get_td_returns(self, rewards: np.ndarray | np.ma.MaskedArray, observations: np.ndarray | np.ma.MaskedArray) -> np.ndarray | np.ma.MaskedArray:
        """ Compute TD(λ) returns """
        isMasked = isinstance(rewards, np.ma.MaskedArray)
        returns = np.empty_like(rewards)
        values = torch2np(self.baseline.forward(np2torch(observations)))
        if isMasked:
            mask = rewards.mask
            rewards = rewards.filled(0)  # we want to add when OR on the masks
            values = values.filled(0)
        returns[:, -1] = rewards[:, -1] + self.discount * values[:, -1]
        for t in reversed(range(rewards.shape[1]-1)):
            returns[:, t] = rewards[:, t] + self.discount * ((1 - self.lambd) * values[:, t+1] + self.lambd * returns[:, t+1])
        if isMasked:
            returns = np.ma.masked_array(returns, mask)
        return returns

    def get_advantages(self, returns, trajectories):
        advantages = self.baseline.calculate_advantage(returns, trajectories)
        return advantages

    def update_policy(self, observations, actions, advantages, logprobs):
        observations = np2torch(observations, True)
        actions      = np2torch(actions, True)
        advantages   = np2torch(advantages, True)
        old_logprobs = np2torch(logprobs, True)
        for arr in [observations, actions, advantages, old_logprobs]:
            print(arr._masked_mask.requires_grad, arr._masked_data.requires_grad)
        
        distribution = self.policy.action_distribution(observations)
        log_probs    = self.policy.log_probs(distribution, actions)
        z_ratio      = torch.exp(log_probs - old_logprobs)
        entropy_loss = self.policy.entropy(distribution, observations)
        if self.do_clip:
            clip_z   = torch.clip(z_ratio,1-self.eps_clip,1+self.eps_clip)
            minimum  = torch.min(z_ratio*advantages,clip_z*advantages)
        else:
            minimum  = z_ratio * advantages
        self.optimizer.zero_grad()
        print(minimum._masked_mask.requires_grad, entropy_loss._masked_mask.requires_grad)
        loss         = -torch.mean(minimum) - torch.mean(self.entropy_coef * entropy_loss)
        loss.backward()
        self.optimizer.step()

v = np.ma.empty((nb, nt, val_dim))
v[:] = np.ma.masked
r = np.ma.empty((nb, nt))
r[:] = np.ma.masked
for i in range(nb):
    length = int(np.random.rand(1)[0]*nt)
    v[i, :length, :] = np.random.rand(length, val_dim)
    r[i, :length] = np.random.rand(length)

a = v[:,:,:act_dim]
o = v[:,:,:obs_dim]
policy = PolicyGradient()
P = policy.policy
print(isinstance(o, np.ma.MaskedArray), isinstance(a, np.ma.MaskedArray))
sampled, logprobs = P.act(o, return_log_prob=True)
print(isinstance(sampled, np.ma.MaskedArray), isinstance(logprobs, np.ma.MaskedArray))
returns = policy.get_td_returns(r, v)
finals = policy.get_finals(returns)
advantages = policy.get_advantages(returns, v)
policy.update_policy(o, a, advantages, logprobs)

True True


AttributeError: 'tuple' object has no attribute 'dim'